The 4 attacks defined in the paper are:
1. fixed amplitude fixed frequency sinusoidal signal
2. binarized version of the first
3. frequency hopping sinusoidal signal
4. binarized version of third
5. adding a gausian noise

In [1]:
import os
print(os.path.abspath("TESTING DOC_CV_page-0001.jpg"))


/home/khushi/TESTING DOC_CV_page-0001.jpg


In [6]:
from PIL import Image
import numpy as np

image_path = "/home/khushi/Desktop/CV_Printer_Identification/Testing_images/TESTING_CV.jpg"
image = Image.open(image_path)

gray_array = np.array(image)

bit_depth = gray_array.dtype
image_mode = image.mode
image_shape = gray_array.shape

print(bit_depth, image_mode, image_shape)

uint8 RGB (1650, 1275, 3)


In [14]:
def save_image(array, name):
    img = Image.fromarray(np.clip(array, 0, 255).astype(np.uint8))
    img.save(name + ".jpg")

#Attack 1
def attack_fixed_freq(image, A, f, Rp):
    height, width = image.shape
    s1 = (A/2) * (1 + np.cos(2 * np.pi * f * np.arange(height)/Rp))
    s1 = s1[:, np.newaxis]
    attacked = np.where(image == 0, image+s1, 255)
    return attacked

#Attack 2
def attack_fixed_freq_binarized(image, A, f, Rp):
    height, width = image.shape
    s2 = (1/16) * (1 + np.cos(2 * np.pi * f * np.arange(height)/Rp))
    s2 = s2[:, np.newaxis]
    rand = np.random.rand(height, width)  # this is a random number for thresholding
    attacked = np.where((image == 0) & (rand < s2), 255, image)
    return attacked

#Attack 3
def attack_freq_hopping(image, A, Rp):
    height, width = image.shape
    phi = np.zeros(height)
    f_values = []
    alpha_values = []
    
    # Generate frequency segments
    i = 0
    while i < height:
        # Randomly select frequency and duration
        f = np.random.randint(30, 121)  # Frequency in [30,120]
        alpha = np.random.randint(0, 101)  # Duration in [0,100] rows
        alpha = max(1, alpha)  
        
        # Apply same frequency for alpha rows
        segment_end = min(i + alpha, height)
        for row in range(i, segment_end):
            if row == 0:
                phi[row] = 0      #phi(0) = 0 
            else:
                phi[row] = phi[row-1] + (2 * np.pi * f) / Rp
        
        i = segment_end
    
    s3 = (A / 2) * (1 + np.cos(phi))
    s3 = s3[:, np.newaxis]  
    attacked = np.where(image == 0, image + s3, 255)
    return attacked

#Attack 4
def attack_freq_hopping_binarized(image, Rp, nu=4):
    height, width = image.shape
    phi = np.zeros(height)
    f_values = []
    alpha_values = []
    
    # Generate frequency segments (same as Attack 3)
    i = 0
    while i < height:
        f = np.random.randint(30, 121)
        alpha = np.random.randint(0, 101)
        alpha = max(1, alpha)
        
        segment_end = min(i + alpha, height)
        for row in range(i, segment_end):
            if row == 0:
                phi[row] = 0
            else:
                phi[row] = phi[row-1] + (2 * np.pi * f) / Rp
        
        i = segment_end
    
    s4 = (1 / (2**nu)) * (1 + np.cos(phi))
    s4 = s4[:, np.newaxis]
    rand = np.random.rand(height, width)
    attacked = np.where((image == 0) & (rand < s4), 255, image)
    return attacked

#Attack 5
def attack_gaussian_noise(image, sigma = 5):
    height, width = image.shape
    noise = np.random.normal(3 * sigma/2, sigma, size=(height, width))
    attacked = np.where(image == 0, image+noise, image)
    return attacked


In [15]:
image_path = "/home/khushi/Desktop/CV_Printer_Identification/Testing_images/TESTING_CV.jpg"
og_img = Image.open(image_path).convert("L")
gray = np.array(og_img)

#setting the parameters
A = 20
f = 5
Rp = 600   
""" 
I have one doubt here Rp is a scaling factor that maintain the consistenncy between the digital attack and printing
So should Rp will vary acc to teh printer for each case or it will be fixed accoridn to scanner as finally it is goign to be scanned at last
"""

att1 = attack_fixed_freq(gray, A, f, Rp)
att2 = attack_fixed_freq_binarized(gray, A, f, Rp)
att3 = attack_freq_hopping(gray, A, Rp)
att4 = attack_freq_hopping_binarized(gray, Rp)
att5 = attack_gaussian_noise(gray)

save_image(att1, "attack_fixed_freq")
save_image(att2, "attack_fixed_freq_binarized")
save_image(att3, "attack_freq_hopping")
save_image(att4, "attack_freq_hopping_binarized")
save_image(att5, "attack_gaussian_noise")